In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import  DecisionTreeRegressor
from scipy import stats


In [2]:
#data train
file_train='blogfeedback/train/blogData_train.csv'
train=pd.read_csv(file_train, header=None)

#data test
file_test='blogfeedback/test/blogData_test.csv'
test=pd.read_csv(file_test, header=None)


C:\Users\Tuan Ngua\AppData\Local\Temp\ipykernel_1448\3134575670.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test=pd.read_csv(file_test, header=None)


In [3]:
train.shape, test.shape

((52397, 281), (7625, 281))

Visualization 

In [4]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Set the size and style of the plots
# sns.set_theme(style="whitegrid")
# plt.figure(figsize=(15, 10))

# # Define the feature indices based on the description
# features = {
#     "Total comments before basetime (51)": 50,
#     "Comments in last 24 hours (52)": 51,
#     "Comments between T1 and T2 (53)": 52,
#     "Comments in first 24 hours after publication (54)": 53,
#     "Difference of Attribute 52 and 53 (55)": 54,
#     "Target: Comments in next 24 hours (281)": 280
# }

# # Create histograms and boxplots
# fig, axes = plt.subplots(3, 2, figsize=(15, 15))
# for ax, (label, index) in zip(axes.flatten(), features.items()):
#     sns.histplot(train[index], bins=30, ax=ax, kde=False, color='skyblue')
#     ax.set_title(label)
#     ax.set_ylabel('Frequency')

# # Tight layout for better spacing
# plt.tight_layout()

# # Display the figure
# plt.show()

# # Create scatter plots for relationships
# plt.figure(figsize=(12, 6))
# for i, (label, index) in enumerate(features.items()):
#     if "Target" not in label:
#         plt.scatter(train[index], train[280], alpha=0.5, label=label)
# plt.title('Scatter plot of features vs. Target (Comments in next 24 hours)')
# plt.xlabel('Feature values')
# plt.ylabel('Comments in next 24 hours')
# plt.legend()
# plt.show()


In [5]:
train = train.drop(train.columns[:50], axis=1)  # Drop the first 50 columns
test = test.drop(test.columns[:50], axis=1)  # Drop the first 50 columns

# Split features and labels
X_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

In [6]:
# Print shapes to confirm
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (52397, 230)
y_train shape: (52397,)
X_test shape: (7625, 230)
y_test shape: (7625,)


In [7]:


class RandomForestRegressor():
    """Implement Random Forest regressor from scratch using Decision Tree."""
    
    def __init__(
        self,
        n_estimators=100,
        criterion='mse', 
        max_depth=None,
        min_samples_leaf=1,
        max_features='sqrt', 
        min_impurity_decrease=0.0,
        random_state=0
    ):
        """
        Some important parameters in Random Forest.
        
        Args:
            n_estimators (int): The number of trees in the forest.
            criterion (str): The function to measure the quality of a split ('mse' for mean squared error).
            max_depth (int): The maximum depth of the tree.
            min_samples_leaf (int): The minimum number of samples required to be at a leaf node.
            max_features (str): The number of features to consider when looking for the best split; 'sqrt' for square root.
            min_impurity_decrease (float): A node will be split if this split induces a decrease of the impurity greater than or equal to this value.
            random_state (int): Controls randomness of the bootstrap samples and the features.
        """
        self.n_estimators = n_estimators
        self.criterion =  criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.min_impurity_decrease = min_impurity_decrease
        self.random_state = random_state
        
    def fit(self, X, y):
        """Fit the random forest model."""
        self.n_samples, self.n_features = X.shape
        if self.max_features == 'sqrt':
            self.max_feature = int(np.sqrt(self.n_features))
        
        self.trees = []
        for i in range(self.n_estimators):
            X_train, _, y_train, _ = train_test_split(
                X, 
                y, 
                test_size=0.3, 
                random_state=self.random_state + i
            )
            tree = DecisionTreeRegressor(
                criterion = self.criterion,
                max_depth = self.max_depth,
                min_samples_leaf = self.min_samples_leaf,
                max_features = self.max_features,
                random_state = self.random_state
            )
            tree.fit(X_train, y_train)
            self.trees.append(tree)
    
    def predict(self, X_test):
        """Predict continuous values for X_test."""
        predictions = np.array([tree.predict(X_test) for tree in self.trees])
        predicted_values = np.mean(predictions, axis=0)
        return predicted_values


In [8]:
# Initialize and fit model
clf = RandomForestRegressor(
    n_estimators=100,
    criterion='squared_error', 
    max_depth=10,
    min_samples_leaf=1,
    max_features='sqrt', 
    min_impurity_decrease=0.0,
    random_state=0
)

In [9]:
clf.fit(X_train, y_train)

In [10]:
y_pred=clf.predict(X_test)

In [11]:
# Print the sum of NaN values to confirm their presence
print("NaN values in y_test before:", y_test.isna().sum())

# Replace NaN with 1
y_test.fillna(1, inplace=True)

# Verify the replacement
print("NaN values in y_test after:", y_test.isna().sum())

print(y_test)


NaN values in y_test before: 1
NaN values in y_test after: 0
0       4.0
1       0.0
2       1.0
3       5.0
4       0.0
       ... 
7620    0.0
7621    0.0
7622    3.0
7623    0.0
7624    1.0
Name: 280, Length: 7625, dtype: float64


In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5  # Root Mean Squared Error


print("Mean Squared Error:", mse)
print(f'RMSE : {rmse}')

Mean Squared Error: 637.1497870124964
RMSE : 25.241826142585175


In [13]:
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')

Mean Absolute Error (MAE): 6.677369345384818


In [14]:
import joblib

joblib.dump(clf,'random_forest_regressor.joblib')

['random_forest_regressor.joblib']